In [1]:
from huggingface_hub import login

login()

In [2]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

model_name = "meta-llama/Llama-3.2-3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 확인할 입력들
inputs_list = ["1", "01", "001", "0001"]

for inp in inputs_list:
    # 토크나이징
    tokens = tokenizer(inp, return_tensors="pt")

    # 토큰 ID를 실제 토큰으로 변환
    tokenized_tokens = tokenizer.convert_ids_to_tokens(tokens["input_ids"].squeeze().tolist())

    # 출력용
    print("입력 문자열:", inp)
    print("토크나이즈 결과 (input_ids):", tokens["input_ids"].tolist())
    print("토크나이즈 결과 (토큰들):", tokenized_tokens)

    # 모델에 입력하여 임베딩 추출
    with torch.no_grad():
        outputs = model(**tokens)

    # 첫 번째 토큰의 히든 상태 벡터 사용
    embedding = outputs.last_hidden_state[:, 0, :].squeeze(0).cpu().numpy()

    # 임베딩 출력
    print("임베딩 벡터 (첫 번째 토큰):", embedding[:5])
    print("-" * 50)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

입력 문자열: 1
토크나이즈 결과 (input_ids): [[128000, 16]]
토크나이즈 결과 (토큰들): ['<|begin_of_text|>', '1']
임베딩 벡터 (첫 번째 토큰): [ 0.47611216 -0.37273955  0.44221842  0.2527937   1.4558876 ]
--------------------------------------------------
입력 문자열: 01
토크나이즈 결과 (input_ids): [[128000, 1721]]
토크나이즈 결과 (토큰들): ['<|begin_of_text|>', '01']
임베딩 벡터 (첫 번째 토큰): [ 0.47611216 -0.37273955  0.44221842  0.2527937   1.4558876 ]
--------------------------------------------------
입력 문자열: 001
토크나이즈 결과 (input_ids): [[128000, 4119]]
토크나이즈 결과 (토큰들): ['<|begin_of_text|>', '001']
임베딩 벡터 (첫 번째 토큰): [ 0.47611216 -0.37273955  0.44221842  0.2527937   1.4558876 ]
--------------------------------------------------
입력 문자열: 0001
토크나이즈 결과 (input_ids): [[128000, 931, 16]]
토크나이즈 결과 (토큰들): ['<|begin_of_text|>', '000', '1']
임베딩 벡터 (첫 번째 토큰): [ 0.47611216 -0.37273955  0.44221842  0.2527937   1.4558876 ]
--------------------------------------------------


In [3]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
import pandas as pd

# 모델 및 토크나이저 불러오기
model_name = "meta-llama/Llama-3.2-3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 저장할 숫자 리스트 생성
numbers = (
    [str(i) for i in range(10)] +      # "0" ~ "9"
    [f"{i:02d}" for i in range(100)] + # "00" ~ "99"
    [f"{i:03d}" for i in range(1000)]  # "000" ~ "999"
)

# 숫자별 임베딩 저장 리스트
embedding_list = []

# 숫자별 임베딩 추출
for num in numbers:
    # 토큰 변환
    inputs = tokenizer(num, return_tensors="pt")

    # 모델에 입력하여 임베딩 추출
    with torch.no_grad():
        outputs = model(**inputs)

    # 첫 번째 토큰의 히든 상태 벡터 사용
    embedding = outputs.last_hidden_state[:, 1, :].squeeze(0).cpu().numpy()

    # 데이터 저장 (리스트로 변환 후 추가)
    embedding_list.append([num] + embedding.tolist())

# 데이터프레임 생성 및 CSV 저장
embedding_dim = embedding.shape[0]
columns = ["key"] + [f"dim_{i+1}" for i in range(embedding_dim)]
df = pd.DataFrame(embedding_list, columns=columns)
df.to_csv("llama_embeddings.csv", index=False)

# 저장된 데이터 확인
print("CSV 파일 저장 완료")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

CSV 파일 저장 완료
